# Execution time

We first compare **performance** (execution time):
- of the following CNN **models** (net + weights):
<!--  - GoogleNet; -->
  - AlexNet;
<!--  - SqueezeNet 1.0; -->
<!--  - SqueezeNet 1.1; -->

- when using the following OpenCL BLAS **libraries**:
<!--   - CLBlast dev (> 0.9.0); -->
<!--   - CLBlast dev (> 0.9.0) with Mali-optimised overlay. -->
  - clBLAS 2.4.

## Includes

### Standard

In [ ]:
import os
import sys
import json

### Scientific

In [ ]:
import IPython as ip
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mp

In [ ]:
print 'IPython version: %s' % ip.__version__
print 'NumPy version: %s' % np.__version__
print 'SciPy version: %s' % sp.__version__
print 'Pandas version: %s' % pd.__version__
print 'Matplotlib version: %s' % mp.__version__

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
# import scipy.stats as st

### Collective Knowledge

In [ ]:
import ck.kernel as ck
print 'CK version: %s' % ck.__version__

In [ ]:
r=ck.access({'action':'show', 'module_uoa':'env', 'tags':'tool,opencl,dvdt,prof'})
if r['return']>0:
    print ("Error: %s" % r['error'])
    exit(1)
# Get path the first returned environment entry.
dvdt_prof_dir=r['lst'][0]['meta']['env']['CK_ENV_TOOL_DVDT_PROF']
dvdt_prof_src_python=os.path.join(dvdt_prof_dir,'src','python')
sys.path.append(dvdt_prof_src_python)
import prof_wrangler as pw
pw.test()

## Access experimental results

In [ ]:
def get_experimental_results(tags):
    r=ck.access({'action':'search', 'module_uoa':'experiment', 'tags':tags})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    # FIXME: For now, assume a single entry per the given tags.
    r=ck.access({'action':'list_points', 'module_uoa':'experiment', 'data_uoa': r['lst'][0]['data_uoa']})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    
    results = {}
    for point in r['points']:
        with open(os.path.join(r['path'], 'ckp-%s.0001.json' % point)) as point_file:
            point_data_raw = json.load(point_file)
            run_info_list = [
                characteristics['run']
                for characteristics in point_data_raw['characteristics_list']
                if characteristics['run']['run_success'] == 'yes'
            ]
            # Select characteristics of interest. TODO: simplify.
            point_data_dict = {
                'time_fw_ms' : [ run_info.get('time_fw_ms',0)      for run_info in run_info_list ],
                'dvdt_prof'  : [ run_info.get('dvdt_prof',{})      for run_info in run_info_list ],
                'per_layer'  : [ run_info.get('per_layer_info',[]) for run_info in run_info_list ]
            }
            batch_size = point_data_raw['choices']['env']['CK_CAFFE_BATCH_SIZE']
            results[batch_size] = point_data_dict
    return results

In [ ]:
def get_min_time_fw_per_image(results):
    time_fw_ms = { k : v['time_fw_ms'] for (k,v) in results.iteritems()}
    df = pd.DataFrame(data=time_fw_ms)
    df.index.name = 'repetition'
    df.columns.name = 'batch size'
    return (df.describe().ix['min'] / range(1,len(time_fw_ms)+1)).min()

### AlexNet

In [ ]:
alexnet_clblas_tags = 'time,caffemodel,alexnet,clblas'
alexnet_clblas_results = get_experimental_results(alexnet_clblas_tags)
alexnet_clblas_min_time_fw_per_image = get_min_time_fw_per_image(alexnet_clblas_results)
alexnet_clblas_min_time_fw_per_image

In [ ]:
# dvdt_prof_alexnet_clblas_tags = 'dvdt-prof-,caffemodel,alexnet,clblas'
# dvdt_prof_alexnet_clblas_results = get_experimental_results(dvdt_prof_alexnet_clblas_tags)
# dvdt_prof_alexnet_clblas_min_time_fw_per_image = get_min_time_fw_per_image(dvdt_prof_alexnet_clblas_results)
# dvdt_prof_alexnet_clblas_min_time_fw_per_image

In [ ]:
dvdt_prof_cjson_alexnet_clblas_tags = 'dvdt-prof-cjson,caffemodel,alexnet,clblas'
dvdt_prof_cjson_alexnet_clblas_results = get_experimental_results(dvdt_prof_cjson_alexnet_clblas_tags)
dvdt_prof_cjson_alexnet_clblas_min_time_fw_per_image = get_min_time_fw_per_image(dvdt_prof_cjson_alexnet_clblas_results)
dvdt_prof_cjson_alexnet_clblas_min_time_fw_per_image

In [ ]:
# alexnet_clblast_development_tags = 'time,caffemodel,alexnet,clblast,vdevelopment'
# alexnet_clblast_development_results = get_experimental_results(alexnet_clblast_development_tags)
# alexnet_clblast_development_min_time_fw_per_image = get_min_time_fw_per_image(alexnet_clblast_development_results)
# alexnet_clblast_development_min_time_fw_per_image

In [ ]:
# alexnet_clblast_mali_overlay_tags = 'time,caffemodel,alexnet,clblast,vmali-overlay'
# alexnet_clblast_mali_overlay_results = get_experimental_results(alexnet_clblast_mali_overlay_tags)
# alexnet_clblast_mali_overlay_min_time_fw_per_image = get_min_time_fw_per_image(alexnet_clblast_mali_overlay_results)
# alexnet_clblast_mali_overlay_min_time_fw_per_image

### SqueezeNet 1.0

In [ ]:
# squeezenet_1_0_clblas_tags = 'time,caffemodel,squeezenet-1.0,clblas'
# squeezenet_1_0_clblas_results = get_experimental_results(squeezenet_1_0_clblas_tags)
# squeezenet_1_0_clblas_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_0_clblas_results)
# squeezenet_1_0_clblas_min_time_fw_per_image

In [ ]:
# squeezenet_1_0_clblast_development_tags = 'time,caffemodel,squeezenet-1.0,clblast,vdevelopment'
# squeezenet_1_0_clblast_development_results = get_experimental_results(squeezenet_1_0_clblast_development_tags)
# squeezenet_1_0_clblast_development_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_0_clblast_development_results)
# squeezenet_1_0_clblast_development_min_time_fw_per_image

In [ ]:
# squeezenet_1_0_clblast_mali_overlay_tags = 'time,caffemodel,squeezenet-1.0,clblast,vmali-overlay'
# squeezenet_1_0_clblast_mali_overlay_results = get_experimental_results(squeezenet_1_0_clblast_mali_overlay_tags)
# squeezenet_1_0_clblast_mali_overlay_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_0_clblast_mali_overlay_results)
# squeezenet_1_0_clblast_mali_overlay_min_time_fw_per_image

### SqueezeNet 1.1

In [ ]:
# squeezenet_1_1_clblas_tags = 'time,caffemodel,squeezenet-1.1,clblas'
# squeezenet_1_1_clblas_results = get_experimental_results(squeezenet_1_1_clblas_tags)
# squeezenet_1_1_clblas_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_1_clblas_results)
# squeezenet_1_1_clblas_min_time_fw_per_image

In [ ]:
# squeezenet_1_1_clblast_development_tags = 'time,caffemodel,squeezenet-1.1,clblast,vdevelopment'
# squeezenet_1_1_clblast_development_results = get_experimental_results(squeezenet_1_1_clblast_development_tags)
# squeezenet_1_1_clblast_development_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_1_clblast_development_results)
# squeezenet_1_1_clblast_development_min_time_fw_per_image

In [ ]:
# squeezenet_1_1_clblast_mali_overlay_tags = 'time,caffemodel,squeezenet-1.1,clblast,vmali-overlay'
# squeezenet_1_1_clblast_mali_overlay_results = get_experimental_results(squeezenet_1_1_clblast_mali_overlay_tags)
# squeezenet_1_1_clblast_mali_overlay_min_time_fw_per_image = get_min_time_fw_per_image(squeezenet_1_1_clblast_mali_overlay_results)
# squeezenet_1_1_clblast_mali_overlay_min_time_fw_per_image

### GoogleNet

In [ ]:
# googlenet_clblas_tags = 'time,caffemodel,googlenet,clblas'
# googlenet_clblas_results = get_experimental_results(googlenet_clblas_tags)
# googlenet_clblas_min_time_fw_per_image = get_min_time_fw_per_image(googlenet_clblas_results)
# googlenet_clblas_min_time_fw_per_image

In [ ]:
# googlenet_clblast_development_tags = 'time,caffemodel,googlenet,clblast,vdevelopment'
# googlenet_clblast_development_results = get_experimental_results(googlenet_clblast_development_tags)
# googlenet_clblast_development_min_time_fw_per_image = get_min_time_fw_per_image(googlenet_clblast_development_results)
# googlenet_clblast_development_min_time_fw_per_image

In [ ]:
# googlenet_clblast_mali_overlay_tags = 'time,caffemodel,googlenet,clblast,vmali-overlay'
# googlenet_clblast_mali_overlay_results = get_experimental_results(googlenet_clblast_mali_overlay_tags)
# googlenet_clblast_mali_overlay_min_time_fw_per_image = get_min_time_fw_per_image(googlenet_clblast_mali_overlay_results)
# googlenet_clblast_mali_overlay_min_time_fw_per_image

## Execution time - data frame

In [ ]:
data = {
    'AlexNet w/ cBLAS v2.4' : {
        'w/o dvdt-prof'      : alexnet_clblas_min_time_fw_per_image,
#        'w/ dvdt-prof'       : dvdt_prof_alexnet_clblas_min_time_fw_per_image,
        'w/ dvdt-prof-cjson' : dvdt_prof_cjson_alexnet_clblas_min_time_fw_per_image,
    },
}
df = pd.DataFrame(data)
df

## Execution time - plot

In [ ]:
df.T \
    .plot(title='Execution time per image (ms)',
          kind='bar', rot=0, ylim=[0,1100], figsize=[12, 6], grid=True, legend=True, colormap=cm.autumn)

# Execution profile

In [ ]:
dvdt_prof_cjson_alexnet_clblas_batchsize1_trace0 = dvdt_prof_cjson_alexnet_clblas_results[1]['dvdt_prof'][0]
trace = dvdt_prof_cjson_alexnet_clblas_batchsize1_trace0
trace = pw.index_calls(trace)

In [ ]:
def df_index(nqs, label_fn):
    index = pd.MultiIndex.from_tuples(
        names=('call label', 'call index', 'kernel name'),
        tuples=[
            (label_fn(nq), nq['call_index'], nq['name'])
            for nq in nqs
        ]
    )
    return index

In [ ]:
def df_data(nqs):
    data = [
        {
            'p1 - p0' : nq['profiling']['submit'] - nq['profiling']['queued'], # command queueing time
            'p2 - p1' : nq['profiling']['start']  - nq['profiling']['submit'], # job queueing time
            'p3 - p2' : nq['profiling']['end']    - nq['profiling']['start'],  # kernel execution time
            't1 - t0' : pw.ts_delta_ns(ts_end=nq['timestamp']['end'], ts_start=nq['timestamp']['start']), # chrono time
            't1 - t0 - (p3 - p0)' : 
                        pw.ts_delta_ns(ts_end=nq['timestamp']['end'], ts_start=nq['timestamp']['start']) - 
                        nq['profiling']['end'] + nq['profiling']['queued'],  # chrono overhead
            'gws0' : nq['gws'][0],
            'gws1' : nq['gws'][1],
            'gws2' : nq['gws'][2],
            'lws0' : nq['lws'][0],
            'lws1' : nq['lws'][1],
            'lws2' : nq['lws'][2]
        }
        for nq in nqs
    ]
    return data

In [ ]:
kernel_enqueues = pw.filter_calls(trace, ['clEnqueueNDRangeKernel'])
max_nq_index = kernel_enqueues[-1]['call_index']
max_nq_index_digits = len(str(max_nq_index))

In [ ]:
df_kernel_enqueues = pd.DataFrame(
    data = df_data(
        kernel_enqueues
    ),
    index = df_index(
        kernel_enqueues,
        label_fn = lambda nq: '%s %s' % (str(nq['call_index']).zfill(max_nq_index_digits), nq['name'])
    )
)
df_kernel_enqueues[['p1 - p0', 'p2 - p1', 'p3 - p2', 't1 - t0', 't1 - t0 - (p3 - p0)']] *= 1e-6 # ns -> ms
# df_kernel_enqueues

In [ ]:
df_kernel_enqueues.info(memory_usage=True)

In [ ]:
df_kernel_enqueues.sortlevel('call label')

In [ ]:
df_kernel_enqueues[['t1 - t0']].sum()

In [ ]:
df_kernel_enqueues[['p3 - p2']].sum()

In [ ]:
# Since gws[2] is always 1, summing it gives the number of kernel enqueues.
time_and_num_kernel_enqueues = df_kernel_enqueues[['p3 - p2', 'gws2']].groupby(level='kernel name').sum()
# Update the column and index labels.
time_and_num_kernel_enqueues.columns = ['** Execution time (ms) **', '** Number of enqueues **']
time_and_num_kernel_enqueues.index.name = '** Kernel name **'
time_and_num_kernel_enqueues